# Target SQL Business Case Analysis
This notebook explores e-commerce order data from Target, Brazil.

## 📁 Dataset Overview
- 100K orders from 2016–2018
- 8 tables: orders, order_items, customers, payments, products, sellers, reviews, geolocation

## 🔍 Basic Data Exploration

In [ ]:

-- Data types from customers table
SELECT column_name, data_type 
FROM `zeta-infusion-427205-k4.Target.INFORMATION_SCHEMA.COLUMNS` 
WHERE table_name = 'customers';


In [ ]:

-- Order date range
SELECT MIN(order_purchase_timestamp) AS start_date, MAX(order_purchase_timestamp) AS end_date 
FROM `zeta-infusion-427205-k4.Target.orders`;


In [ ]:

-- Count of customers by city and state
SELECT COUNT(DISTINCT o.customer_id) AS order_count, c.customer_state, c.customer_city 
FROM `zeta-infusion-427205-k4.Target.customers` c
JOIN `zeta-infusion-427205-k4.Target.orders` o ON c.customer_id = o.customer_id
GROUP BY c.customer_state, c.customer_city
ORDER BY order_count DESC;


## 📈 Trend & Seasonality Analysis

In [ ]:

-- Yearly and monthly order trends
SELECT EXTRACT(YEAR FROM o.order_purchase_timestamp) AS year,
       EXTRACT(MONTH FROM o.order_purchase_timestamp) AS month,
       COUNT(DISTINCT o.order_id) AS order_count
FROM `zeta-infusion-427205-k4.Target.orders` o
JOIN `zeta-infusion-427205-k4.Target.customers` c ON o.customer_id = c.customer_id
GROUP BY year, month
ORDER BY year, month;


In [ ]:

-- Monthly seasonality
SELECT EXTRACT(MONTH FROM order_purchase_timestamp) AS month,
       COUNT(DISTINCT order_id) AS order_count
FROM `zeta-infusion-427205-k4.Target.orders`
GROUP BY month
ORDER BY month;


In [ ]:

-- Time-of-day order distribution
SELECT CASE 
         WHEN EXTRACT(HOUR FROM order_purchase_timestamp) BETWEEN 0 AND 6 THEN 'Dawn'
         WHEN EXTRACT(HOUR FROM order_purchase_timestamp) BETWEEN 7 AND 12 THEN 'Morning'
         WHEN EXTRACT(HOUR FROM order_purchase_timestamp) BETWEEN 13 AND 18 THEN 'Afternoon'
         ELSE 'Night'
       END AS period,
       COUNT(order_id) AS order_count
FROM `zeta-infusion-427205-k4.Target.orders`
GROUP BY period
ORDER BY order_count DESC;


## 🚚 Delivery Time & Freight Analysis

In [ ]:

-- Average delivery time by state
SELECT c.customer_state,
       ROUND(AVG(DATE_DIFF(order_delivered_customer_date, order_purchase_timestamp, DAY)), 2) AS avg_delivery_days
FROM `zeta-infusion-427205-k4.Target.orders` o
JOIN `zeta-infusion-427205-k4.Target.customers` c ON o.customer_id = c.customer_id
WHERE order_delivered_customer_date IS NOT NULL
GROUP BY c.customer_state
ORDER BY avg_delivery_days DESC;


In [ ]:

-- Fastest vs slowest delivery states (compared to estimated)
SELECT c.customer_state,
       ROUND(AVG(DATE_DIFF(order_estimated_delivery_date, order_delivered_customer_date, DAY)), 2) AS early_delivery_days
FROM `zeta-infusion-427205-k4.Target.orders` o
JOIN `zeta-infusion-427205-k4.Target.customers` c ON o.customer_id = c.customer_id
GROUP BY c.customer_state
ORDER BY early_delivery_days ASC
LIMIT 5;


## 💳 Payment Behavior

In [ ]:

-- Orders by payment type monthly
SELECT p.payment_type,
       EXTRACT(MONTH FROM o.order_purchase_timestamp) AS month,
       COUNT(DISTINCT o.order_id) AS order_count
FROM `zeta-infusion-427205-k4.Target.orders` o
JOIN `zeta-infusion-427205-k4.Target.payments` p ON o.order_id = p.order_id
GROUP BY p.payment_type, month
ORDER BY p.payment_type, month;


In [ ]:

-- Orders by number of installments
SELECT p.payment_installments,
       COUNT(DISTINCT o.order_id) AS order_count
FROM `zeta-infusion-427205-k4.Target.orders` o
JOIN `zeta-infusion-427205-k4.Target.payments` p ON o.order_id = p.order_id
WHERE o.order_status <> 'canceled'
GROUP BY p.payment_installments
ORDER BY order_count DESC;


## 🌟 Review Scores & Customer Insights

In [ ]:

-- Review scores by state (pivot style)
SELECT * 
FROM (
  SELECT c.customer_state, orv.review_score 
  FROM `target.order_reviews` orv 
  JOIN `target.orders` o USING(order_id) 
  JOIN `target.customers` c USING(customer_id)
) 
PIVOT(COUNT(*) FOR review_score IN (1, 2, 3, 4, 5));


## 📌 Actionable Insights & Recommendations


- ~1.2% of orders were canceled or unavailable – needs investigation.
- Peak shopping times: **Afternoon & Night** – adjust workforce shifts.
- **3 states dominate** order volume – growth opportunity in others.
- Delivery time is a possible barrier in low-order regions – optimize logistics.
- Negative reviews are clustered in specific states – analyze service quality.
